# Imports

In [1]:
from datetime import datetime
import pandas as pd
from string import Template
from tqdm import tqdm
import random
import string
import os

# DOI generator

In [2]:
alphabet = string.ascii_uppercase + string.digits

def random_choice():
    return ''.join(random.choices(alphabet, k=8))

# Variables

In [ ]:
file = 'EC ISTVS2024_2024-11-22_1732286348.xlsx' #Path to excel file from Easy Chair

batch_id = 12344
depositor_name = 'hello@istvs.org:istvs'
email_address = 'hello@istvs.org'
registrant = 'International Society of Terrain-Vehicle Systems'

conference_name = '16th European-African Regional Conference of the ISTVS'
conference_theme = 'Terramechanics'
conference_acronym = 'ISTVS2023'
conference_sponsor = ' '
conference_number = 16
conference_location = 'Lublin, Poland'
start_month="10"
start_year="2023"
start_day="11"
end_year="2023"
end_month="10"
end_day="13"

proceedings_title = 'Proceedings of the 16th European-African Regional Conference of the ISTVS'
proceedings_subject = 'Terramechanics'

publisher_name = 'International Society for Terrain-Vehicle Systems'
publisher_place = 'Hanover, NH, U.S.'

publication_date_year = 2023
isbn = '978-1-942112-55-6'

processings_doi = '10.56884/IGQO3531' # '10.56884/' + random_choice()
processings_resource = 'http://2023.istvs.org'

article_year = 2023

DOI_prefix = '10.56884'

URL_GITBOOK = 'https://2023.istvs.org/submissions/papers/'

In [4]:
processings_doi

'10.56884/H3WLZCGW'

In [5]:
now = datetime.now()
ts = now.strftime("%Y%m%d%H%M%S")

# Confersion from Easy chair format

In [6]:
if(os.path.isfile('formatted_'+file)):
    print(f"The formatted file for {file} already exists. Further processing will be performed without making changes to this file (formatted_{file}).")
else:
    df_EC = pd.read_excel(file)

    df_EC = df_EC[df_EC['submission_type'] == 'full_paper_sub']
    
    df_EC['Authors'] = df_EC['Authors'].str.replace('\n', ' ')
    df_EC['Title'] = df_EC['Title'].str.replace('\n', ' ').str.title()
    df_EC['Keywords'] = df_EC['Keywords'].str.replace('\\n', ', ', regex=True)
    df_EC['Abstract'] = df_EC['Abstract'].str.replace('\n', ' ')
    
    df_EC['URL - GITBOOK'] = df_EC['#'].apply(lambda x: URL_GITBOOK+str(x).zfill(4))

    splitted = df_EC['Authors'].str.replace(' and ',', ').str.split(', ', expand=True)
    
    new_columns = []
    for i in range((splitted.shape[1])):
        new_columns.append(f'Author{i+1}')
    splitted.columns = new_columns

    dois = []
    for i in range(df_EC.shape[0]):
        dois.append(DOI_prefix+'/'+random_choice())

    df_doi = pd.DataFrame(dois, columns=['DOI'])
    
    df_EC = df_EC.reset_index(drop=True)
    splitted = splitted.reset_index(drop=True)
    
    df_excel = pd.concat([df_EC[['#','Authors','Title','Keywords','Abstract','URL - GITBOOK']], df_doi, splitted], axis=1)

    df_excel.to_excel('formatted_'+file)

# GitBook pages

In [7]:
def genMD(EC_id, Title, Authors, DOI, Abstract, conference_name, conference_acronym):
    with open('gitbook.template.txt', 'r') as f:
        
        if ', ' in Authors:
            Authors = Authors.replace(' and ',', and ')
        
        author_info = {
            'id': str(EC_id).zfill(4),
            'Title': Title,
            'conference_name': conference_name,
            'DOI': DOI,
            'conference_acronym': conference_acronym,
            'Authors': Authors,
            'Abstract': Abstract
        }
        src = Template(f.read())
        result = src.substitute(author_info)
        
    f = open('gitbook/'+str(EC_id).zfill(4)+'.md', "w", encoding="utf-8")
    f.write(result)
    f.close()

# DOIs

In [10]:
df = pd.read_excel('formatted_'+file)

papers = ''
for index, row in tqdm(df.iterrows()):
    authors = ''
#if(row['Status'] == 'Ok'):
        
    genMD(row['#'], row['Title'], row['Authors'], row['DOI'], row['Abstract'], conference_name, conference_acronym)
    
    for i in range(1,int(df.columns[-1].replace('Author',''))):
        sequenceText = 'additional'
        if(i == 1): 
            sequenceText = 'first'
        if(not pd.isna(row[f'Author{i}'])):

            names, surname = row[f'Author{i}'].rsplit(' ', 1)

            author_info = {
                'seq': sequenceText,
                'fname': names,
                'lname': surname
            }
            with open('author.template.txt', 'r') as f:
                src = Template(f.read())
                result = src.substitute(author_info)
                authors += result

    if(authors != ''):
        paper_info = {
            'authors': authors,
            'title': row['Title'],
            'year': article_year,
            'doi': row['DOI'],
            'url': row['URL - GITBOOK']
            }


        with open('conference_paper.template.txt', 'r') as f:
            src = Template(f.read())
            paper = src.substitute(paper_info)
            papers += paper

50it [00:00, 282.42it/s]


In [11]:
conference_info = {
    'batch_id': batch_id,
    'timestamp': ts,
    'depositor_name': depositor_name,
    'email_address': email_address,
    'registrant': registrant,
    'conference_name': conference_name,
    'conference_theme': conference_theme,
    'conference_acronym': conference_acronym,
    'conference_sponsor': conference_sponsor,
    'conference_number': conference_number,
    'conference_location': conference_location,
    'start_month': start_month,
    'start_year': start_year,
    'start_day': start_day,
    'end_year': end_year,
    'end_month': end_month,
    'end_day': end_day,
    'proceedings_title': proceedings_title,
    'proceedings_subject': proceedings_subject,
    'publisher_name': publisher_name,
    'publisher_place': publisher_place,
    'publication_date_year': publication_date_year,
    'isbn': isbn,
    'processings_doi': processings_doi,
    'processings_resource': processings_resource,
    'conference_papers': papers
}

with open('conference.template.txt', 'r') as f:
    src = Template(f.read())
    result = src.substitute(conference_info)
    with open(f'dois_{ts}.xml', 'x', encoding="utf-8") as f:
        f.write(result)

FileExistsError: [Errno 17] File exists: 'dois_20241123185259.xml'